<a href="https://colab.research.google.com/github/cindysteward/Cindy-Steward-Portfolio/blob/main/ai_text_to_music_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Text-to-Music Generator Using [Mubert API](https://mubert.com/#).**

Here is the code for a TTM generator made using Mubert, using AI generative music. This code was inspired through a Youtube Channel discussing AI projects and a connected tutorial on [GitHub](https://github.com/MubertAI).
After the intial set-up, the code works by 
1. Finding the tags that fit the prompt (through the similar_track function).
2. Generating a track based on the tags with which the prompt is classified.

I personally find these datasets and Mubert can become more accurate. It is fun to do, it is a fun way of generating music using AI, but as the prompts are limited to a certain amount of tags, it falls short in accuracy.

## **Initial Set-Up**
First, I set up the environment by installing the necessary packages and libraries.

In [ ]:
#Instead of creating a method, I just install each one by listing them in 3 commands.
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -U sentence-transformers
!pip install httpx

In [27]:
import numpy as np
from sentence_transformers import SentenceTransformer

from IPython.display import Audio, display
import httpx
import json
import subprocess, time

## **Defining Variables and Functions**

In [8]:
#Necessary variables are defined. The tags are written in strings, that Mubert will use to generate tracks. This variable is used to crate an array containing all of thse tags.
minilm = SentenceTransformer("all-MiniLM-L6-v2")
tags_str = "tribal,action,kids,neo-classic,run 130,pumped,jazz / funk,ethnic,dubtechno,reggae,acid jazz,liquidfunk,funk,witch house,tech house,underground,artists,mystical,disco,sensorium,r&b,agender,psychedelic trance / psytrance,peaceful,run 140,piano,run 160,setting,meditation,christmas,ambient,horror,cinematic,electro house,idm,bass,minimal,underscore,drums,glitchy,beautiful,technology,tribal house,country pop,jazz & funk,documentary,space,classical,valentines,chillstep,experimental,trap,new jack swing,drama,post-rock,tense,corporate,neutral,happy,analog,funky,spiritual,sberzvuk special,chill hop,dramatic,catchy,holidays,fitness 90,optimistic,orchestra,acid techno,energizing,romantic,minimal house,breaks,hyper pop,warm up,dreamy,dark,urban,microfunk,dub,nu disco,vogue,keys,hardcore,aggressive,indie,electro funk,beauty,relaxing,trance,pop,hiphop,soft,acoustic,chillrave / ethno-house,deep techno,angry,dance,fun,dubstep,tropical,latin pop,heroic,world music,inspirational,uplifting,atmosphere,art,epic,advertising,chillout,scary,spooky,slow ballad,saxophone,summer,erotic,jazzy,energy 100,kara mar,xmas,atmospheric,indie pop,hip-hop,yoga,reggaeton,lounge,travel,running,folk,chillrave & ethno-house,detective,darkambient,chill,fantasy,minimal techno,special,night,tropical house,downtempo,lullaby,meditative,upbeat,glitch hop,fitness,neurofunk,sexual,indie rock,future pop,jazz,cyberpunk,melancholic,happy hardcore,family / kids,synths,electric guitar,comedy,psychedelic trance & psytrance,edm,psychedelic rock,calm,zen,bells,podcast,melodic house,ethnic percussion,nature,heavy,bassline,indie dance,techno,drumnbass,synth pop,vaporwave,sad,8-bit,chillgressive,deep,orchestral,futuristic,hardtechno,nostalgic,big room,sci-fi,tutorial,joyful,pads,minimal 170,drill,ethnic 108,amusing,sleepy ambient,psychill,italo disco,lofi,house,acoustic guitar,bassline house,rock,k-pop,synthwave,deep house,electronica,gabber,nightlife,sport & fitness,road trip,celebration,electro,disco house,electronic"
tags = np.array(tags_str.split(","))
tags_emb = minilm.encode(tags)

In [25]:
#This function finds similar, used in the tags_fitting_prompt, scoring similarity to identify fitting tags.
def similar_track(em, embs, method='cosine'):
  scores = []
  for ref in embs:
        if method == 'cosine': 
            scores.append(1 - np.dot(ref, em)/(np.linalg.norm(ref)*np.linalg.norm(em)))
        if method == 'norm': 
            scores.append(np.linalg.norm(ref - em))
  return np.array(scores), np.argsort(scores)

In [22]:
#Using similar_track, this function uses the score to rank top tags, linking tags to the given prompt.
def tags_fitting_prompt(prompts, top_n=3, debug=False):
  prompts_embeddings = minilm.encode(prompts)
  ret = []
  for i, pe in enumerate(prompts_embeddings):
      scores, idxs = similar_track(pe, tags_emb)
      top_tags = tags[idxs[:top_n]]
      top_prob = 1 - scores[idxs[:top_n]]
      ret.append((prompts[i], list(top_tags)))
      return ret


In [19]:
#Here, through using the e-mail I have for my account, and using json, I generate my personal access token and define API methods. 
email= "cindy.j.steward@gmail.com"
r = httpx.post('https://api-b2b.mubert.com/v2/GetServiceAccess', 
    json={
        "method":"GetServiceAccess",
        "params": {
            "email": email,
            "license":"ttmmubertlicense#f0acYBenRcfeFpNT4wpYGaTQIyDI4mJGv5MfIhBFz97NXDwDNFHmMRsBSzmGsJwbTpP1A6i07AXcIeAHo5",
            "token":"4951f6428e83172a4f39de05d5b3ab10d58560b8",
            "mode": "loop"
        }
    })

rdata = json.loads(r.text)
pat = rdata['data']['pat']

In [10]:
#A function that generates a track using the tags that were found to fit the prompt. This is the function that uses all above to actually generate the new track/music, using Mubert API.
def generate_track_tags(tags, pat, duration, maxit=20, autoplay=False, loop=False):
    if loop:
      mode = "loop"
    else:
      mode = "track"
    r = httpx.post("https://api-b2b.mubert.com/v2/RecordTrackTTM",
        json={
          "method":"RecordTrackTTM",
          "params": {
              "pat": pat, 
              "duration": duration,
              "tags": tags,
              "mode": mode
          }
      })
    
    rdata = json.loads(r.text)
    assert rdata['status'] == 1, rdata['error']['text']
    trackurl = rdata['data']['tasks'][0]['download_link']
    
    for i in range(maxit):
      r = httpx.get(trackurl)
      if r.status_code == 200:
          display(Audio(trackurl, autoplay=autoplay))
          break

# **Generate a Track!**

In [62]:
prompt = "happy meditation" #Prompt that defines what kind of music you want to generate. The website recommends basing the prompt on a mood, a genre, or an acitivity (like sport, cardio, mediation, etc).
#I found mentioning an artists isn't as accurate because of this code's limitations.
duration = 15 #Set the duration of the track generated in seconds.
loop = False

def generate_track(prompt, duration, loop=False): #This function uses the variables and makes use of generate_track_tags to create a new track. 
  _, prompttags = tags_fitting_prompt([prompt,])[0]
  generate_track_tags(prompttags, pat, duration, autoplay=True, loop=loop)

In [63]:
generate_track(prompt, duration, loop)